### Import libraries.

In [1]:
import sqlite3
import json
import pandas as pd

### Use SQL library of choice. 

Used sqlite since it's saving locally from local runs. Can and will probably switch to SQLAlchemy for database url instance.

In [2]:
# creating file path
dbfile = '/Users/isaacmenchaca/Desktop/hnlMturkTask1/participants.db'
# Create a SQL connection to our SQLite database
con = sqlite3.connect(dbfile)

# creating cursor
cur = con.cursor()

# reading all table names
table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# here is you table list
print(table_list)



[('apscheduler_jobs',), ('assignments',), ('amt_hit',), ('campaign',)]


### Query assignments database.

Contains info per participant. Display using pandas.

In [3]:
# Verify that you can query task data.
df = pd.read_sql_query("SELECT * FROM assignments", con)
display(df)

,uniqueid,assignmentid,workerid,hitid,ipaddress,browser,platform,language,cond,counterbalance,codeversion,beginhit,beginexp,endhit,bonus,status,mode,datastring
0,debugJ72QI6:debug50U9IQ,debug50U9IQ,debugJ72QI6,debug8IFCN4,127.0.0.1,safari,macos,en-us,0,0,0.0.1,2022-01-07 10:32:25.970613,2022-01-07 10:32:32.788966,2022-01-07 10:32:38.747559,0.0,3,debug,"{""condition"": 0, ""counterbalance"": 0, ""assignm..."
1,debugMF614L:debugAO8832,debugAO8832,debugMF614L,debugY5W1G7,127.0.0.1,safari,macos,en-us,0,0,0.0.1,2022-01-07 10:32:47.620495,2022-01-07 10:32:53.758352,2022-01-07 10:32:58.631147,0.0,3,debug,"{""condition"": 0, ""counterbalance"": 0, ""assignm..."
2,debug297THI:debugSGINL0,debugSGINL0,debug297THI,debugUD1L1Y,127.0.0.1,safari,macos,en-us,0,0,0.0.1,2022-01-07 10:33:08.681703,2022-01-07 10:33:16.375084,2022-01-07 10:33:21.070493,0.0,3,debug,"{""condition"": 0, ""counterbalance"": 0, ""assignm..."


### Verify data saved by PsiTurk.

If the data is there, your data saving method using psiturk is functional.

In [18]:
for i, datastring in enumerate(df['datastring'].values):
        print('Participant', i + 1)
        for participantData in json.loads(datastring)['data']:
            print(participantData['trialdata'])
        print()

Participant 1
{'phase': 'INSTRUCTIONS', 'templates': ['instructions/instruct-1.html', 'instructions/instruct-2.html', 'instructions/instruct-practiceStart.html'], 'action': 'Begin'}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-1.html', 'indexOf': 0, 'action': 'NextPage', 'viewTime': 1016}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-2.html', 'indexOf': 1, 'action': 'NextPage', 'viewTime': 612}
{'phase': 'INSTRUCTIONS', 'template': 'instructions/instruct-practiceStart.html', 'indexOf': 2, 'action': 'FinishInstructions', 'viewTime': 726}
{'rt': 701, 'stimulus': '<p style="font-size: 30px;">This is the practice trial. Press any key to begin.</p>', 'response': 'enter', 'trial_type': 'html-keyboard-response', 'trial_index': 0, 'time_elapsed': 706, 'internal_node_id': '0.0-0.0'}
{'rt': 1732, 'answer': 'lessThan50', 'response': 'lessThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 1, 'time_elapsed': 2442, 'internal_node_id': '0.0-1.0'}

### Test custom code for assigning workers bonuses.

Test how you will scrape the data and insert into custom.py code.

In [19]:
for i, datastring in enumerate(df['datastring'].values):
        print('Participant', i + 1)
        bonus = 0
        for participantData in json.loads(datastring)['data']:
            if "trial_type" in participantData['trialdata'].keys():
                if participantData['trialdata']['trial_type'] == 'DisplayTableStim':
                    
                    print(participantData['trialdata'])
                    if participantData['trialdata']['correct']:
                        bonus += 0.02
                        
        print('Participant bonus:', bonus)
        print()

Participant 1
{'rt': 1732, 'answer': 'lessThan50', 'response': 'lessThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 1, 'time_elapsed': 2442, 'internal_node_id': '0.0-1.0'}
{'rt': 1561, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 4, 'time_elapsed': 5866, 'internal_node_id': '0.0-4.0'}
{'rt': 1726, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 5, 'time_elapsed': 7598, 'internal_node_id': '0.0-5.0'}
{'rt': 1411, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 6, 'time_elapsed': 9016, 'internal_node_id': '0.0-6.0'}
{'rt': 1227, 'answer': 'greaterThan50', 'response': 'greaterThan50', 'correct': True, 'trial_type': 'DisplayTableStim', 'trial_index': 7, 'time_elapsed': 10249, 'internal_node_id': '0.0-7.0'}
Participant bonus: 0.1

Participant 

### Close connection to Database.

In [5]:
con.close()